In [3]:
pip install opencv-python tensorflow tensorflow-lite pytesseract pyttsx3 numpy


Note: you may need to restart the kernel to use updated packages.Requirement already satisfied: opencv-python in e:\jupyternotebookprj\prj\myenv\lib\site-packages (4.11.0.86)



ERROR: Could not find a version that satisfies the requirement tensorflow-lite (from versions: none)
ERROR: No matching distribution found for tensorflow-lite
You should consider upgrading via the 'E:\jupyternotebookprj\prj\myenv\Scripts\python.exe -m pip install --upgrade pip' command.


In [4]:
pip install pyttsx3


     -------------------------------------- 241.5/241.5 KB 1.8 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'E:\jupyternotebookprj\prj\myenv\Scripts\python.exe -m pip install --upgrade pip' command.


In [ ]:
import cv2
import numpy as np
import tensorflow as tf
import pyttsx3
from tensorflow.lite.python.interpreter import Interpreter

# Initialize text-to-speech engine
engine = pyttsx3.init()

# Function to load labels from the coco_labels.txt file
def load_labels(label_file):
    """Load class labels from a text file."""
    labels = {}
    with open(label_file, 'r') as file:
        for i, line in enumerate(file.readlines()):
            labels[i] = line.strip()  # Map index to label name
    return labels

# Load the labels from coco_labels.txt file
LABELS = load_labels('models/coco_labels.txt')

# Load the TensorFlow Lite model
interpreter = Interpreter(model_path="models/coco_ssd_mobilenet_v1_1.0_quant.tflite")
interpreter.allocate_tensors()

# Initialize webcam
cap = cv2.VideoCapture(0)

def get_distance_from_camera(focal_length, known_width, pixel_width):
    """
    Calculate the distance from the camera based on focal length, known object width, and pixel width of object.
    Formula: distance = (focal_length * known_width) / pixel_width
    """
    if pixel_width == 0:
        return 0
    return (focal_length * known_width) / pixel_width

def detect_objects(frame):
    # Resize the frame to match the input shape of the model (300x300 for this model)
    input_tensor = cv2.resize(frame, (300, 300))
    input_tensor = np.expand_dims(input_tensor, axis=0)
    input_tensor = np.array(input_tensor, dtype=np.uint8)

    # Get input and output tensor details
    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()

    # Set input tensor
    interpreter.set_tensor(input_details[0]['index'], input_tensor)

    # Run the interpreter
    interpreter.invoke()

    # Get the output
    boxes = interpreter.get_tensor(output_details[0]['index'])[0]  # Bounding boxes
    classes = interpreter.get_tensor(output_details[1]['index'])[0]  # Class ids
    scores = interpreter.get_tensor(output_details[2]['index'])[0]  # Scores
    num_objects = int(interpreter.get_tensor(output_details[3]['index'])[0])  # Number of detected objects

    return boxes, classes, scores, num_objects

def alert_object(detected_objects):
    for object in detected_objects:
        label = object["label"]
        distance = object["distance"]
        if distance <= 0.60:  # Trigger alert if distance is less than or equal to 0.50 meters
            alert_text = f"Warning: {label} ahead at {distance:.2f} meters!"
            print(alert_text)  # For debugging
            engine.say(alert_text)  # Text-to-speech
            engine.runAndWait()

def main():
    # Focal length and known width of an object (e.g., person)
    focal_length = 700  # This is an example; you'd need to calibrate it
    known_width = 0.5  # Example: average width of a person in meters
    
    while True:
        ret, frame = cap.read()
        if not ret:
            print("Failed to grab frame")
            break

        # Object detection
        boxes, classes, scores, num_objects = detect_objects(frame)

        detected_objects = []

        for i in range(num_objects):
            if scores[i] > 0.5:  # Confidence threshold
                ymin, xmin, ymax, xmax = boxes[i]
                xmin, ymin, xmax, ymax = int(xmin * frame.shape[1]), int(ymin * frame.shape[0]), int(xmax * frame.shape[1]), int(ymax * frame.shape[0])

                object_class = int(classes[i])
                label = LABELS.get(object_class, 'unknown')

                # Calculate pixel width (for distance calculation)
                pixel_width = xmax - xmin
                distance = get_distance_from_camera(focal_length, known_width, pixel_width)
                
                detected_objects.append({"label": label, "distance": distance})

                # Draw bounding box (optional, if you want to see the detection)
                cv2.rectangle(frame, (xmin, ymin), (xmax, ymax), (0, 255, 0), 2)
                cv2.putText(frame, f"{label} {distance:.2f}m", (xmin, ymin - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

        # Give voice alerts
        alert_object(detected_objects)

        # Show the frame with detection (optional)
        cv2.imshow('Object Detection', frame)

        # Break on pressing 'q'
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    main()
